In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Movie_rec_sys").getOrCreate()

df = spark.read.csv("Data\movielens_ratings.csv", inferSchema = True, header = True)

# observe structure and some data
df.printSchema()
print("rows:",df.count(), "  cols:", len(df.columns))
df.show(5)
df.describe().show()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)

rows: 1501   cols: 3
+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [14]:
# Import model
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# We can do a split to evaluate how well our model performed, but it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

# Split
(training, test) = df.randomSplit([0.8, 0.2])

# Build model
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [16]:
# predictions
predictions = model.transform(test)
predictions.show(5)

# evaluation:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    19|   2.277198|
|     31|   1.0|    18| -1.2308383|
|     85|   3.0|     1|  1.6115822|
|     85|   5.0|    16|-0.54863775|
|     85|   1.0|     5|  2.7045946|
+-------+------+------+-----------+
only showing top 5 rows

Root-mean-square error = 1.789821197738708


In [31]:
# simulate a fresh user to test model

single_user = test.filter(test["userId"]==11).select("movieId","userId")
single_user.show()


+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     10|    11|
|     19|    11|
|     20|    11|
|     35|    11|
|     41|    11|
|     47|    11|
|     50|    11|
|     71|    11|
|     79|    11|
|     80|    11|
+-------+------+



In [30]:
recommendations = model.transform(single_user)

recommendations.orderBy("prediction", ascending = False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     10|    11|  2.2102685|
|     80|    11|  1.5329337|
|     50|    11|  1.4884136|
|     79|    11|   1.104497|
|     19|    11| 0.80065566|
|     35|    11|  0.6485561|
|     47|    11| 0.42805374|
|     20|    11|-0.14330846|
|     71|    11|-0.18324454|
|      0|    11|  -0.604515|
|     41|    11| -1.4870186|
+-------+------+-----------+

